# SUBJECT : ABOUT HUBER LOSS
## Optimization with huber loss 


In [ ]:
%matplotlib inline
import numpy as np
from scipy import linalg
import time
import matplotlib.pyplot as plt

## Part 0: Why a robust regression model


Let us consider the problem of regression from $n$ observations
$x_i \in \mathbb{R}^{p}$,
$1 \leq i \leq n$. We aim to learn a function:
$$f: x \in \mathbb{R}^{p}\mapsto y\in\mathbb{R}$$
from the $n$ annotated training samples $(x_{i},y_{i})$ supposed i.i.d. from an unknown probability distribution on $\mathbb{R}^p \times \mathbb{R}$. Once this function is learnt, it will be possible to use it to predict the label $y$ associated to a new sample $x$.

The types of model we consider in this project are so-called *robust models* that can deal with samples corrupted by strong artifacts.

Let's generate such a dataset in 1D to illustrate the problem when using the squared loss ($\|\cdot\|^2$).

In [ ]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression

# Generate toy data
X, y = make_regression(n_samples=20, n_features=1, random_state=0,
                       noise=4.0, bias=10.0)

# Add an outlier
X[0, 0] = 2.
y[0] = 350

# Fit the model
reg = LinearRegression()
reg.fit(X, y)

# Visualize the model
x = X[:, 0]
y_pred = reg.coef_ * x + reg.intercept_

plt.plot(x, y, 'b.')
plt.plot(x, y_pred, 'g-', label="linear regression")
plt.xlabel("X")
plt.ylabel("y")
plt.legend(loc=0)
plt.show()

<div class="alert alert-success">
    <b>QUESTION 0:</b>
     <ul>
       <li>Describe the issue you observe and suggest an explanation and a possible solution.</li>
    </ul>
</div>

INSERT YOUR ANSWER HERE


## Huber Loss

One version of the Huber function ($H_\epsilon : \mathbb{R} \rightarrow \mathbb{R}$) reads:

$$
    H_\epsilon (x) = \left\{
	\begin{aligned}
	x^2 & \quad \mathrm{ if } \quad |x| < \epsilon \\
    2 \epsilon |x| - \epsilon^2 & \quad \mathrm{ otherwise }
	\end{aligned}
    \right.
$$

Working in a regression setting, the Huber loss between 2 targets $y$ and $y'$ reads:

$$
    \mathcal{L}(y, y') = H_\epsilon (y - y')
$$

Here is an implemention of the Huber function:

In [ ]:
epsilon = 1.

## fill in 
def huber(x, epsilon=epsilon):


<div class="alert alert-success">
    <b>QUESTION 1:</b>
     <ul>
       <li>Plot the Huber function vs. the squared function ($x \rightarrow x^2$) vs. the absolute value function ($x \rightarrow |x|$) between -3 and 3 using $\epsilon = 1$</li>
    </ul>
</div>

In [ ]:
### FILL IN 


<div class="alert alert-success">
    <b>QUESTION 2:</b>
     <ul>
       <li>Justify the convexity of the Huber function as defined above.</li>
    </ul>
</div>

INSERT YOUR ANSWER HERE

we have: $H_\varepsilon(X) =\begin{cases}x^2 & \mid x\mid   <  \varepsilon\\2\varepsilon\mid x\mid - \varepsilon^2 & otherwise\end{cases} $

so: $H'_\varepsilon(X) =\begin{cases}2x & \mid x\mid   <  \varepsilon\\2\varepsilon sign(x)  &  otherwise\end{cases}$

$\Rightarrow H''_\varepsilon(X) = \begin{cases}2 & \mid x\mid   <  \varepsilon\\0  &  otherwise\end{cases}$


This means that $ 0\leq H''_\varepsilon(X) \leq 2$. So the Huber function is convex and 2-smooth. As a result, we propose the value 2 for the Lipschitz constant of the gradient.

<div class="alert alert-success">
    <b>QUESTION 3:</b>
     <ul>
       <li>Write a function that computes the gradient of the Huber loss.</li>
    </ul>
</div>

**Remark:** You will use the `scipy.optimize.check_grad` function to assess the validity of your result. You will need to test your gradient in both the linear and quadratic regions of the Huber function (not just in one location).

In [ ]:
### FILL IN

Let us define the cost function associated to the empirical risk with some regularization function $\mathcal{R}$:

$$
    (\mathcal{P}_{f,\mathcal{R}}):
	\begin{aligned}
	\min_{w \in \mathbb{R}^p, b \in \mathbb{R}} \quad \frac{1}{n} \sum_{i=1}^n f(y_i - x_i^\top w - b) + \lambda \mathcal{R}(w) \enspace ,
	\end{aligned}
$$

where $f$ is a scalar function defining the loss (Huber, squared, absolute etc.). The variable $b$ is the bias or intercept term.

In [ ]:
from scipy.optimize import fmin_l_bfgs_b

lbda = 0.01

def pobj_l2(params, X=X, y=y, lbda=lbda, epsilon=epsilon):
    w = params[1:]
    b = params[0]
    return np.mean(huber(y - np.dot(X, w) - b, epsilon=epsilon)) + lbda * np.sum(w ** 2)

def grad_huber_l2(params,X=X, y=y, lbda=lbda ,epsilon=epsilon):
    w = params[1:]
    b = params[0]
    grad_w = (-np.mean(X.T@grad_huber(y - np.dot(X, w) - b, epsilon=epsilon))+ 2*lbda*w)
    grad_b = -np.mean(grad_huber(y - np.dot(X, w) - b, epsilon=epsilon))
    return np.concatenate([grad_b, grad_w], axis=None)

def huber_lbfgs_l2(X=X, y=y, lbda=lbda, epsilon=epsilon):
    w0 = np.zeros(X.shape[1]+1)
    params,_,_  = fmin_l_bfgs_b(pobj_l2, w0, grad_huber_l2)
    return params


# FILL IN  (for visualization)

